# From _main.ipynb_

##### Basic imports

In [4]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
# from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import pandas as pd
import numpy as np 

In [5]:
import pandas as pd

def encode_text_colum(df, text_column, vectorizer): 
    """
    Encodes a text column with the given vectorizer, drops the old column (with text),
    and returns the dataframe with the encoded text.

    Args:
        df (pd.DataFrame): The dataframe.
        text_column (str): The name of the text column to be encoded.
        vectorizer: The text vectorizer.

    Returns:
        pd.DataFrame: The dataframe with the encoded text.
    """
    # Vectorize the text column
    df_vectorized = vectorizer.fit_transform(df[text_column])
    
    # Create a dataframe from the vectorized data
    df_encoded = pd.DataFrame(df_vectorized.toarray(), columns=[f"{text_column}_{i}" for i in range(df_vectorized.shape[1])])

    # Drop the original text column
    df.drop([text_column], axis=1, inplace=True)

    # Concatenate the original dataframe with the encoded text dataframe
    df = pd.concat([df, df_encoded], axis=1)

    return df

# Example usage:
# vectorizer = TfidfVectorizer(max_features=1000)
# X_train_ab = encode_text_column(X_train, 'abstract', vectorizer)
# X_test_ab = encode_text_column(X_test, 'abstract', vectorizer)
# X_train_de = encode_text_column(X_train_ab, 'description_text', vectorizer)
# X_test_de = encode_text_column(X_test_ab, 'description_text', vectorizer)


##### Loading the dataframe

In [6]:
df = pd.read_csv('data/modelready_220423.csv')

##### Preprocessing

In [32]:
# extract unique countries in the df
unique_values = set()
df['countries_in_family'].apply(lambda x: unique_values.update(x.strip("[]").replace("'", "").split())) 

# Create new columns for each unique value
for value in unique_values:
    # each country has a column (1 if the patent belong to the country 0 otherwise)
    df[value] = df['countries_in_family'].apply(lambda x: 1 if value in x else 0)


In [8]:
df = df[df.abstract.notna()].copy() # drop all samples without abstract

In [33]:
# encode company names
df['company_name_encoded'] = df.company_name.astype('category').cat.codes  # encode companies

# remove non-numeric columns
df_columns_dropped = df.drop(['publication_number', 'company_name', 'countries_in_family', 'publn_nr',
       'primary_cpc'], axis = 1)


# f0_ has the same value as commercialization, the other two shouldn't be used
df_columns_dropped = df_columns_dropped.drop(['f0_', 'centrality', 'similarity'], axis = 1)

In [34]:
### PROBLEMATIC CELL ###
# WHAT SHOULD I DO HERE ???????

# remove text as I can't compute min and max on it
text = df_columns_dropped[['abstract', 'description_text']] # putting them aside for later


df_columns_dropped.drop(['abstract', 'description_text'], axis=1, inplace=True)

print(f'missing values = {df_columns_dropped.isna().sum().sum()} ')# some missin values
df_no_missing = df_columns_dropped.fillna(df_columns_dropped.mean()).copy()
print(f'missing values after filling= {df_no_missing.isna().sum().sum()} ')

missing values = 88097 
missing values after filling= 0 


In [35]:
# putting text back in
#df_no_missing[['abstract', 'description_text']] = text  


# extracting what we'll try to predict
y = df_no_missing['commercialized']
df_no_missing.drop('commercialized', axis= 1, inplace=True)

# dropping columns where all the value are the same (min = max) they would be zero if I apply min max rescaling
min_eq_max = df_no_missing.columns[df_no_missing.min() == df_no_missing.max()].to_list()
df_clean = df_no_missing.drop(min_eq_max, axis=1)

# putting text back in
df_clean[['abstract', 'description_text']] = text 


In [36]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(df_clean, y, test_size=0.20, random_state=42)

#rescale 
# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [37]:
print(X_train.isna().sum().sum() / (X_train.shape[0] * X_train.shape[1]))

0.0


In [38]:
# bag of words for abstract
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
X_train_ab = encode_text_colum(X_train, 'abstract', vectorizer)
X_test_ab = encode_text_colum(X_test, 'abstract', vectorizer)

In [39]:
print(X_train_ab.isna().sum().sum() / (X_train_ab.shape[0] * X_train_ab.shape[1]))
print(X_test_ab.isna().sum().sum() / (X_test_ab.shape[0] * X_test_ab.shape[1]))

0.25519205390011807
0.45341488277268094


In [40]:
# Replace NaN values with zeros in X_train_ab
X_train_ab.fillna(0, inplace=True)

# Replace NaN values with zeros in X_test_ab
X_test_ab.fillna(0, inplace=True)


In [41]:
print(X_train_ab.isna().sum().sum() / (X_train_ab.shape[0] * X_train_ab.shape[1]))
print(X_test_ab.isna().sum().sum() / (X_test_ab.shape[0] * X_test_ab.shape[1]))

0.0
0.0


In [42]:
# Train a Multinomial Naive Bayes classifier
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train_ab, y_train)

# Make predictions on the test set
y_pred = naive_bayes_model.predict(X_test_ab)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report_str)

ValueError: could not convert string to float: 'FIELD OF THE INVENTION \n     The invention relates to an improved enteral nutritional formula and more particularly to infant formulas which contain ribo-nucleotide equivalents at a level of at least 10 mg/100 Kcal of formula and wherein the ribo-nucleotide components are at specific ratios. \n     BACKGROUND OF THE INVENTION \n     The composition of human milk serves as a valuable reference for improving infant formula. However, human milk contains living cells, hormones, active enzymes, immunoglobulins and components with unique molecular structures that cannot be replicated in infant formula. Unlike human milk, infant formula must remain stable on the shelf for up to thirty-six (36) months. These fundamental differences between human milk and infant formula often mandate differences in the composition to achieve similar clinical outcome. \n     Human milk has served as a valuable reference for improving infant formula. The investigat

In [30]:
# bag of words for description_text
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
X_train_de = encode_text_colum(X_train_ab, 'description_text', vectorizer)
X_test_de = encode_text_colum(X_test_ab, 'description_text', vectorizer)

AttributeError: 'int' object has no attribute 'lower'

In [ ]:

df_no_missing = df_columns_dropped.fillna(df_columns_dropped.mean()).copy()

# extracting what we'll try to predict
y = df_no_missing['commercialized']
df_no_missing.drop('commercialized', axis= 1, inplace=True)



# dropping columns where all the value are the same (min = max) they would be zero if I apply min max rescaling
min_eq_max = df_no_missing.columns[df_no_missing.min() == df_no_missing.max()].to_list()
df_clean = df_no_missing.drop(min_eq_max, axis=1)


In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(df_clean, y, test_size=0.20, random_state=42)

#rescale 
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# bag of words for abstract
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
X_train_ab = encode_text_colum(X_train, 'abstract', vectorizer)
X_test_ab = encode_text_colum(X_test, 'abstract', vectorizer)

# bag of words for description_text
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
X_train_de = encode_text_colum(X_train_ab, 'description_text', vectorizer)
X_test_de = encode_text_colum(X_test_ab, 'description_text', vectorizer)


# Naive Bayes Model

In [60]:
# bag of words for description_text
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
X_train_de = encode_text_colum(X_train_de, 'description_text', vectorizer)
X_test_de = encode_text_colum(X_test_de, 'description_text', vectorizer)

# Convert feature names to strings
X_train_de.columns = X_train_de.columns.astype(str)
X_test_de.columns = X_test_de.columns.astype(str)

# Train Naive Bayes model
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train_de, y_train)

# Make predictions on the test set
predictions = naive_bayes_model.predict(X_test_de)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Display the results
print(f"Naive Bayes Accuracy: {accuracy:.2f}")
print("Naive Bayes Classification Report:")
print(report)


NameError: name 'X_train_de' is not defined

In [8]:
# bag of words for description_text
vectorizer = TfidfVectorizer(max_features=1000)  # Adjust 'max_features' as needed
X_train_de = encode_text_colum(X_train_ab, 'description_text', vectorizer)
X_test_de = encode_text_colum(X_test_ab, 'description_text', vectorizer)

# Convert feature names to strings
X_train_de.columns = X_train_de.columns.astype(str)
X_test_de.columns = X_test_de.columns.astype(str)

# Train Naive Bayes model
naive_bayes_model = MultinomialNB()
naive_bayes_model.fit(X_train_de, y_train)

# Make predictions on the test set
predictions = naive_bayes_model.predict(X_test_de)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

# Display the results
print(f"Naive Bayes Accuracy: {accuracy:.2f}")
print("Naive Bayes Classification Report:")
print(report)


KeyError: 'description_text'